<a href="https://colab.research.google.com/github/DCI-alxogm/ml2023-starmath4/blob/main/Oct20-21/DESI_Varios_archivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install astroML

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 3.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
from astropy.io import fits #el fits nos permite leer archivos con extension fits. El formato .fits es un archivo binario que almacena conjuntos de datos mediante arreglos numéricos
from astropy.table import Table
import matplotlib.pyplot as plt

In [ ]:
files = ["/content/coadd-sv3-dark-25956.fits","/content/coadd-sv3-dark-25965.fits"]
bflux = np.vstack([fits.open(filex)["B_FLUX"].data for filex in files])
bwave = np.vstack([fits.open(filex)["B_WAVELENGTH"].data for filex in files])

In [ ]:
#Código revisado en https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

#Implementación de PCA para el flujo
from sklearn.decomposition import PCA
pca=PCA(3) #con esto indico que buscaremos los 3 mejores parámetros
pca.fit(bflux) #Buscamos los parámetros que nos dan más información del flujo


PCA(n_components=3)

In [ ]:
#Datos adicionales del análisis de PCA
vr=pca.explained_variance_ratio_
sv=pca.singular_values_

print("Varianza explicada: ",vr)
print("Valores singulares: ",sv)

In [ ]:
zcat=Table.read("/content/zall-pix-edr-vac.fits",hdu=1)
zcat

ValueError: ignored

In [ ]:
zcat=zcat['TARGETID','SURVEY','SPECTYPE','Z'] #redefinimos la tabla usando sólo los parámetros que nos interesan
zcat

In [ ]:
mask=zcat["SURVEY"]=='sv3' #filtramos "survey" para sólo considerar los que tengan sv3
zcat=zcat[mask]
zcat

In [ ]:
mask=np.in1d(zcat["TARGETID"],tids)
zcat=zcat[mask]
zcat

In [ ]:
lya=1216 #Lyman-alpha emission
#graficar el corrimiento al rojo de los primeros 10 targetid

for i,tid in enumerate(zcat["TARGETID"][:10]):
  z=zcat["Z"][i]
  indx=np.where(tids==tid)[0][0]
  plt.plot(bwave,bflux[indx])
  plt.axvline(1216*(1+z),color='r')
  plt.show()

No pude hacer la extrapolacion debido a que no me cargo bien el ultimo archivo, pero a lo que consulte, sería algo así, según lo que consulté en https://docs.scipy.org/doc/scipy/tutorial/interpolate.html:

from scipy import interpolate

#basicamente pasamos al marco en reposo usando las transformaciones del redshifting, para poder usar luego el scipy e interpolate en el arreglo auxiliar
min_redshift = np.min(zcat["Z"])
max_redshift = np.max(zcat["Z"])
#uso el factor 1/(1+z) para convertir entre marcos de referencia
min_wavelength = bwave[0] / (1 + max_redshift)  
max_wavelength = bwave[-1] / (1 + min_redshift)  
restwavelength = np.linspace(min_wavelength, max_wavelength,10000)

aux_interpolacion=[]
for i, tid in enumerate(zcat["TARGETID"]):
    z = zcat["Z"][i]
    indx = np.where(tid == tids)[0][0]
    wavel = bwave / (1 + z)
    aux_interpolacion = interpolate.interp1d(wavel, bflux[indx], kind='linear', fill_value="extrapolate")
    aux_interpolacion.append(aux_interpolacion)
print(aux_interpolacion)